In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

$\large \text{Consider the function:} h(x)=∑_{i=1}^N \frac{(x_i-2^i)^2}{8^i} $

$\large \text{Let N =n so:} \\ h(x)=∑_{i=1}^n \frac{(x_i-2^i)^2}{8^i} \\ ⇒h(x)=∑_{i=1}^n (\frac{x_i^2}{8^i}+\frac{(-x_i)}{2^{2i-1}}+\frac{1}{2^i}) \\ \text{We can write above function in form of } \ x^ΤAx+2b^Τx+c \\ \text{where} \ x= \begin{bmatrix} 
	x_1 & x_2 & x_3 .... & x_n \\
	\end{bmatrix}^Τ \\ A= \text{Diagonal matrix with diagonal entries }\ \{ {\frac{1}{2^3}},\frac{1}{2^6},\frac{1}{2^9},.,.,.,,.,\frac{1}{2^{3*n}} \} \\ b=\begin{bmatrix} 
	\frac{-1}{2^1} & \frac{-1}{2^3} & \frac{-1}{2^5} .... & \frac{-1}{2^{2*n-1}} \\
	\end{bmatrix} \\ c= ∑_{i=1}^n\frac{1}{2^i}$

$ So for N=3  \\ h(x)=∑_{i=1}^3 (\frac{x_i^2}{8^i}+\frac{(-x_i)}{2^{2i-1}}+\frac{1}{2^i})  $

In [19]:
def evalf(x):
  assert len(x)==3 and type(x) is np.ndarray
  return sum((x[i]-2**(i+1))**2/(8**(i+1)) for i in range(3) )

In [20]:
def evalg(x):
  assert len(x)==3 and type(x) is np.ndarray
  return np.array([(x[i-1])/(2**(3*i-1))-1/(2**(2*i-1)) for i in range(1,4)])

In [21]:
def compute_steplength_exact(gradf, A,x): #add appropriate arguments to the function 
  assert type(gradf) is np.ndarray and len(gradf) == 3 
  assert type(A) is np.ndarray and A.shape[0] == 3 and  A.shape[1] == 3 #allow only a 2x2 array
   
  numerator=np.dot(evalg(x).transpose(),evalg(x))    
  denominator=2*np.dot(np.dot(evalg(x).transpose(),A),evalg(x))
  step_length=numerator/denominator

  
  return step_length

In [22]:
#Complete the module to compute the steplength by using the backtracking line search
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 3 
  assert type(gradf) is np.ndarray and len(gradf) == 3 
  
  alpha = alpha_start
  p=-gradf
  while evalf(x+alpha*p)> evalf(x)+gamma*alpha*(np.dot(evalg(x).transpose(),p)):
    alpha=alpha*rho
  return alpha

In [23]:
#we define the types of line search methods that we have implemented
EXACT_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH = 2

In [24]:
def find_minimizer(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 3 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  # construct a suitable A matrix for the quadratic function 
  A = np.array([[1/8, 0,0],[0,1/64,0],[0,0,1/512]])
  x = start_x
  g_x = evalg(x)

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)

  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
  
    if line_search_type == EXACT_LINE_SEARCH:
      step_length = compute_steplength_exact(g_x, A,x) #call the new function you wrote to compute the steplength
      #raise ValueError('EXACT LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) #call the new function you wrote to compute the steplength
      #raise ValueError('BACKTRACKING LINE SEARCH NOT YET IMPLEMENTED')
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point

    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x,k 


$\large \text{[R] Note down the minimizer and minimum function value of h(x).} $

In [25]:
print('Let out starting position is [1,1,1] and tolerance value is 10^(-5)')
my_start_x = np.array([1,1,1])
my_tol= 1e-5
print('Solution using closed-form expression')
x_opt_cls,number_of_iter_cls=find_minimizer(my_start_x,my_tol,EXACT_LINE_SEARCH)
print('Minimizer of function:',x_opt_cls)
print('Minuimum function value:',evalf(x_opt_cls))
print('number of iterations:',number_of_iter_cls)
print('---------------------------------------------------------------------')
#check what happens when you call find_minimzer using backtracking line search
print('Solution using backtracking line search')
x_opt_bls,number_of_iter_bls = find_minimizer(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, 1, 0.5,0.5)
print('Minimizer of function:',x_opt_bls)
print('Minuimum function value:',evalf(x_opt_bls))
print('number of iterations:',number_of_iter_bls)

Let out starting position is [1,1,1] and tolerance value is 10^(-5)
Solution using closed-form expression
Minimizer of function: [2.00001649 4.         7.99768767]
Minuimum function value: 1.0477122031231117e-08
number of iterations: 151
---------------------------------------------------------------------
Solution using backtracking line search
Params for Backtracking LS: alpha start: 1 rho: 0.5  gamma: 0.5
Minimizer of function: [2.         4.         7.99744063]
Minuimum function value: 1.2793697352715871e-08
number of iterations: 2022


$\large \text{Answer:The minimizer of function for N=3 is :[2.00001649,4,7.99768767]} ≃ \text{[2,4,8] } \\ $ 

$ \large \text{[R] Consider stopping tolerance} \ τ = 10^{−10} \ \text{and starting point} \ x_0 = ( 1
64 , 1
8 , 1). \\ \text{ Compare the number of iterations
taken by the gradient descent procedure which uses exact step length computation against}  \\ \text{ the gradient descent
procedure which uses the backtracking line search procedure (with} α_0 = 1, ρ = 0.5, γ = 0.5). \\ \text{Comment on your
observations.} $

In [34]:
start_upd=np.array([1/64,1/8,1])
tol_upd=1e-10
alpha_0=1
rho=0.5
gamma=0.5
print('Solution using closed-form expression')
x_opt_cls,number_of_iter_cls=find_minimizer(start_upd,tol_upd,EXACT_LINE_SEARCH)
print('Minimizer of function:',x_opt_cls)
print('Minuimum function value:',evalf(x_opt_cls))
print('number of iterations:',number_of_iter_cls)
print('---------------------------------------------------------------------')
#check what happens when you call find_minimzer using backtracking line search
print('Solution using backtracking line search')
x_opt_bls,number_of_iter_bls = find_minimizer(start_upd,tol_upd, BACKTRACKING_LINE_SEARCH, alpha_0, rho,gamma)
print('Minimizer of function:',x_opt_bls)
print('Minuimum function value:',evalf(x_opt_bls))
print('number of iterations:',number_of_iter_bls)


Solution using closed-form expression
Minimizer of function: [2.         4.         7.99999998]
Minuimum function value: 9.150071377581033e-19
number of iterations: 269
---------------------------------------------------------------------
Solution using backtracking line search
Params for Backtracking LS: alpha start: 1 rho: 0.5  gamma: 0.5
Minimizer of function: [2.         4.         7.99999997]
Minuimum function value: 1.2748574165464873e-18
number of iterations: 4964


$\large \text{Observations:} \\ \text{As we can see than number of iteration taken by the exact line search method is 269 and number of iterations taken by the backtracking} \\ \text{  line search method is 4964 which is far greater than iterations taken by exact line search.As we can conclude than exact line search method} \\ \text{  takes low number of iterations to reach optimal condition.}$

$ \large \text{[R] Check if similar observations hold in the N = 4 case as well. Choose a starting point} \ x_0 = ( 1
512 , 1
64 , 1
8 , 1)
 \\ \text{and for the backtracking line search procedure, use} \  α_0 = 1, ρ = 0.5, γ = 0.5. \\ \text{Comment on your observations.} $

$ So for N=4  \\ h(x)=∑_{i=1}^4 (\frac{x_i^2}{8^i}+\frac{(-x_i)}{2^{2i-1}}+\frac{1}{2^i})  $

In [27]:
def evalf_n4(x):
  assert len(x)==4 and type(x) is np.ndarray
  return sum((x[i]-2**(i+1))**2/(8**(i+1)) for i in range(4) )

In [28]:
def evalg_n4(x):
  assert len(x)==4 and type(x) is np.ndarray
  return np.array([(x[i-1])/(2**(3*i-1))-1/(2**(2*i-1)) for i in range(1,5)])

In [29]:
def compute_steplength_exact_n4(gradf, A,x): #add appropriate arguments to the function 
  assert type(gradf) is np.ndarray and len(gradf) == 4 
  assert type(A) is np.ndarray and A.shape[0] == 4 and  A.shape[1] == 4 #allow only a 2x2 array
   
  numerator=np.dot(evalg_n4(x).transpose(),evalg_n4(x))    
  denominator=2*np.dot(np.dot(evalg_n4(x).transpose(),A),evalg_n4(x))
  step_length=numerator/denominator

  
  return step_length

In [30]:
#Complete the module to compute the steplength by using the backtracking line search
def compute_steplength_backtracking_n4(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 4 
  assert type(gradf) is np.ndarray and len(gradf) == 4 
  
  alpha = alpha_start
  p=-gradf
  while evalf_n4(x+alpha*p)> evalf_n4(x)+gamma*alpha*(np.dot(evalg_n4(x).transpose(),p)):
    alpha=alpha*rho
  return alpha

In [31]:
def find_minimizer_n4(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 4 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  # construct a suitable A matrix for the quadratic function 
  A = np.array([[1/8, 0,0,0],[0,1/64,0,0],[0,0,1/512,0],[0,0,0,1/2**12]])
  x = start_x
  g_x = evalg_n4(x)

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)

  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
  
    if line_search_type == EXACT_LINE_SEARCH:
      step_length = compute_steplength_exact_n4(g_x, A,x) #call the new function you wrote to compute the steplength
      #raise ValueError('EXACT LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_n4(x,g_x, alpha_start,rho, gamma) #call the new function you wrote to compute the steplength
      #raise ValueError('BACKTRACKING LINE SEARCH NOT YET IMPLEMENTED')
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg_n4(x) #compute gradient at new point

    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x,k 


In [33]:
start=np.array([1/512,1/64,1/8,1])
tol=1e-10
alpha_0=1
rho=0.5
gamma=0.5
print('Solution using closed-form expression')
x_opt_cls,number_of_iter_cls=find_minimizer_n4(start,tol,EXACT_LINE_SEARCH)
print('Minimizer of function:',x_opt_cls)
print('Minuimum function value:',evalf_n4(x_opt_cls))
print('number of iterations:',number_of_iter_cls)
print('---------------------------------------------------------------------')
print('Solution using backtracking line search')
x_opt_bls,number_of_iter_bls = find_minimizer_n4(start,tol, BACKTRACKING_LINE_SEARCH, alpha_0, rho,gamma)
print('Minimizer of function:',x_opt_bls)
print('Minuimum function value:',evalf_n4(x_opt_bls))
print('number of iterations:',number_of_iter_bls)


Solution using closed-form expression
Minimizer of function: [ 2.          4.          8.         15.99999981]
Minuimum function value: 8.8565993523583e-18
number of iterations: 2013
---------------------------------------------------------------------
Solution using backtracking line search
Params for Backtracking LS: alpha start: 1 rho: 0.5  gamma: 0.5
Minimizer of function: [ 2.         4.         8.        15.9999998]
Minuimum function value: 1.0237544252113035e-17
number of iterations: 37079


$\large \text{Observations:} \\ \text{As per the above results we came to know that minimizer of function obtained by both methods} \\ \text{  (backtracking-line-search or exact-line-search) is [2,4,8,16] and minimum function value is:} 8.8565993523583e-18 ≃ 0 \\ \text{As we can here also observe that number of iteration taken to reach optimal condition by exact line search method is less than} \\ \text{ number of iterations taken to reach optimal condition by backtracking line search method.Since:} \\ 741<13507 \\ \text{number of iterations by exact line search} < \text{number of iterations by backtracking line search}$

$\large \text{Question[R]:} \\ \text{Can you also comment on the possible observations for N > 4 case as well, without actually running the
program?} $

$\large \text{Answer :} \\ \text{As we can see that minimizers for values of N=3 and N=4 we obtained the following results:} \\ \text{for } N=3: \\ \text{minimizer is:} = [2,4,8]=[2^1,2^2,2^3] \\ \text{minimum function value is: } = 0 \\ \text{for } N=4: \\ \text{minimizer is:} = [2,4,8,16]=[2^1,2^2,2^3,2^4] \\ \text{minimum function value is: } = 0 \\ \text{Hence we can define this for general value of n also by induction :} \\ \text{for } N=n: \\ \text{minimizer is:} = [2,4,8,....,2^n]=[2^1,2^2,2^3,....,2^n] \\ \text{minimum function value is: } = 0 \\ \text{We can also observe that exact line search method to compute step lenght was taking low number of iterations to reach } \\ \text{ optimal condition in comparison of backtracking line search method.}$